# Query Drivers by Page

Use this notebook to understand which queries are driving each page and how concentrated performance is across a few terms.

In [ ]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")

import pandas as pd
import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import build_date_params, default_query_window, get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()
window = default_query_window(config.DEFAULT_DAYS_BACK)
PAGE_PATH = "/"  # update this page path to inspect a specific page

In [ ]:
query = f"""
SELECT
  report_date,
  page_path,
  query,
  SUM(clicks) AS clicks,
  SUM(impressions) AS impressions,
  SAFE_DIVIDE(SUM(clicks), NULLIF(SUM(impressions), 0)) AS ctr,
  SAFE_DIVIDE(SUM(avg_position * impressions), NULLIF(SUM(impressions), 0)) AS avg_position
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.curated_search_query_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
  AND page_path = @page_path
GROUP BY report_date, page_path, query
ORDER BY report_date DESC, clicks DESC
"""

from google.cloud import bigquery

params = build_date_params(window) + [
    bigquery.ScalarQueryParameter("page_path", "STRING", PAGE_PATH),
]

df_queries = run_query(client, query, params=params)
df_queries.head(20)

In [ ]:
top_queries = (
    df_queries.groupby("query", as_index=False)[["clicks", "impressions"]]
    .sum()
    .sort_values("clicks", ascending=False)
    .head(20)
)

total_clicks = float(top_queries["clicks"].sum()) or 1.0
top_queries["click_share"] = top_queries["clicks"] / total_clicks

fig = px.bar(
    top_queries.sort_values("clicks", ascending=True),
    x="clicks",
    y="query",
    orientation="h",
    template="lifeline",
    title=f"Top Query Drivers for {PAGE_PATH}",
)
fig.update_layout(height=700, margin={"l": 320})
lifeline_theme.add_lifeline_logo(fig)
fig.show()

top_queries.head(10)